Huggingface tokenizers of pretrained model such as BERT, ELECTRA, ALBERTO etc returns subword tokens that is not convenient some of problems that is each label assinged to word, not subword. You can do many different solution for this, but in this notebook we will do a solution using $gather$ function that is already implemented all common neural network freamworks. Thanks Mateusz Bednarski for clear explanation of $gather$ function on https://medium.com/analytics-vidhya/understanding-indexing-with-pytorch-gather-33717a84ebc4

This study is coded using Torch but the required function $gather$ and $repeat$ is also avaible for Tensoflow 

In [1]:
import torch

Let's create a dummy embedding tensor, It's dimension is batch_size x max_seq_lenght_according_to_subtoken_level x hidden_dim. You can look the given link for more detail about why created as an dummy tensor

In [2]:
batch_size = 3
max_seq_len = 9
hidden_size = 6
pretrained_embedings_on_subword_level = torch.empty(batch_size, max_seq_len, hidden_size)
for i in range(batch_size):
    for j in range(max_seq_len):
        for k in range(hidden_size):
            pretrained_embedings_on_subword_level[i,j,k] = i + j*10 + k*100

In [3]:
pretrained_embedings_on_subword_level

tensor([[[  0., 100., 200., 300., 400., 500.],
         [ 10., 110., 210., 310., 410., 510.],
         [ 20., 120., 220., 320., 420., 520.],
         [ 30., 130., 230., 330., 430., 530.],
         [ 40., 140., 240., 340., 440., 540.],
         [ 50., 150., 250., 350., 450., 550.],
         [ 60., 160., 260., 360., 460., 560.],
         [ 70., 170., 270., 370., 470., 570.],
         [ 80., 180., 280., 380., 480., 580.]],

        [[  1., 101., 201., 301., 401., 501.],
         [ 11., 111., 211., 311., 411., 511.],
         [ 21., 121., 221., 321., 421., 521.],
         [ 31., 131., 231., 331., 431., 531.],
         [ 41., 141., 241., 341., 441., 541.],
         [ 51., 151., 251., 351., 451., 551.],
         [ 61., 161., 261., 361., 461., 561.],
         [ 71., 171., 271., 371., 471., 571.],
         [ 81., 181., 281., 381., 481., 581.]],

        [[  2., 102., 202., 302., 402., 502.],
         [ 12., 112., 212., 312., 412., 512.],
         [ 22., 122., 222., 322., 422., 522.],
         

Let's create a dummy offset tensor. It's dimension is batch_size x max_seq_according_to_token_level. We ignore some of subtokens, so we expect that max_seq_according_to_token_level<max_seq_lenght_according_to_subtoken_level x hidden_dim. We will get the first subtoken embeding here, you can modify it according to your request such as last subtoken, mean of subtoken etc. Note that if you want to get mean of subtoken embedings as a token embedding, so you may need also other fuctions such as $RaggedTensor$ (in Tensorflow) etc.

In [4]:
token_start_indexes = torch.tensor([[0,1,4,6,7],
                                   [0,1,2,3,8],
                                   [0,1,3,4,6]])

Below code lines do our aim successfully but it takes a bit time. We want to make it using $gather$ function to speed up.

In [5]:
x=pretrained_embedings_on_subword_level[:, token_start_indexes]
li=[x[i][i] for i,batch_i in enumerate(x)]
pretrained_embedings_on_word_level_ref = torch.stack(li,dim=0)

In [6]:
pretrained_embedings_on_word_level_ref

tensor([[[  0., 100., 200., 300., 400., 500.],
         [ 10., 110., 210., 310., 410., 510.],
         [ 40., 140., 240., 340., 440., 540.],
         [ 60., 160., 260., 360., 460., 560.],
         [ 70., 170., 270., 370., 470., 570.]],

        [[  1., 101., 201., 301., 401., 501.],
         [ 11., 111., 211., 311., 411., 511.],
         [ 21., 121., 221., 321., 421., 521.],
         [ 31., 131., 231., 331., 431., 531.],
         [ 81., 181., 281., 381., 481., 581.]],

        [[  2., 102., 202., 302., 402., 502.],
         [ 12., 112., 212., 312., 412., 512.],
         [ 32., 132., 232., 332., 432., 532.],
         [ 42., 142., 242., 342., 442., 542.],
         [ 62., 162., 262., 362., 462., 562.]]])

Now we can check your solution is right or not with this reference

To use gather we have to aware three parameter, 
- input : input tensor
- dim     : dimension along to collect values
- indices : tensor with indices of values to collect (it needs to be same dimension with input tensor except dim axis)

In [7]:
token_start_indexes.shape

torch.Size([3, 5])

In [8]:
pretrained_embedings_on_subword_level.shape

torch.Size([3, 9, 6])

We will collect our vector on dim 1, we will get the vector for the first subtokens 

In [9]:
#repeat 6 times
batch_size, max_seq_according_to_token_level = token_start_indexes.shape 
indices = token_start_indexes.repeat(1,6).view(batch_size,-1,max_seq_according_to_token_level).transpose(2,1)

In [10]:
indices.shape

torch.Size([3, 5, 6])

In [11]:
indices

tensor([[[0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1],
         [4, 4, 4, 4, 4, 4],
         [6, 6, 6, 6, 6, 6],
         [7, 7, 7, 7, 7, 7]],

        [[0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1],
         [2, 2, 2, 2, 2, 2],
         [3, 3, 3, 3, 3, 3],
         [8, 8, 8, 8, 8, 8]],

        [[0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1],
         [3, 3, 3, 3, 3, 3],
         [4, 4, 4, 4, 4, 4],
         [6, 6, 6, 6, 6, 6]]])

In [12]:
pretrained_embedings_on_word_level = torch.gather(pretrained_embedings_on_subword_level,1,indices)

In [13]:
pretrained_embedings_on_word_level_ref == pretrained_embedings_on_word_level

tensor([[[True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True]],

        [[True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True]],

        [[True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True]]])

Yes, we have done it succesfully.

Let's compare their speed on a huge tensor.

In [14]:
import time

def operation_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# dummy embedding tensor
batch_size = 30000
max_seq_len = 9
hidden_size = 6
pretrained_embedings_on_subword_level = torch.empty(batch_size, max_seq_len, hidden_size)
for i in range(batch_size):
    for j in range(max_seq_len):
        for k in range(hidden_size):
            pretrained_embedings_on_subword_level[i,j,k] = i + j*10 + k*100
# dummy token start index tensor            
token_start_indexes = torch.tensor([[0,1,4,6,7],
                                   [0,1,2,3,8],
                                   [0,1,3,4,6]])
token_start_indexes = token_start_indexes.repeat(1,int(batch_size/3)).view(batch_size,-1)

In [15]:
tic=time.time()
x=pretrained_embedings_on_subword_level[:, token_start_indexes]
li=[x[i][i] for i,batch_i in enumerate(x)]
pretrained_embedings_on_word_level_ref = torch.stack(li,dim=0)
toc=time.time()
mins, secs = operation_time(tic, toc)
print(f'Finished in {mins}m {secs}s without using gather function')

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 108000000000 bytes. Error code 12 (Cannot allocate memory)


Even it could not allocate memory on CPU for such a tensor. Using GPU also will not change the result of this race. Sorry, I could not test on GPU, because it is a bit busy now. 

In [16]:
tic=time.time()
batch_size, max_seq_according_to_token_level = token_start_indexes.shape 
indices = token_start_indexes.repeat(1,6).view(batch_size,-1,max_seq_according_to_token_level).transpose(2,1)
pretrained_embedings_on_word_level = torch.gather(pretrained_embedings_on_subword_level,1,indices)
toc=time.time()
mins, secs = operation_time(tic, toc)
print(f'Finished in {mins}m {secs}s with using gather function')

Finished in 0m 0s with using gather function


In [17]:
pretrained_embedings_on_word_level

tensor([[[0.0000e+00, 1.0000e+02, 2.0000e+02, 3.0000e+02, 4.0000e+02,
          5.0000e+02],
         [1.0000e+01, 1.1000e+02, 2.1000e+02, 3.1000e+02, 4.1000e+02,
          5.1000e+02],
         [4.0000e+01, 1.4000e+02, 2.4000e+02, 3.4000e+02, 4.4000e+02,
          5.4000e+02],
         [6.0000e+01, 1.6000e+02, 2.6000e+02, 3.6000e+02, 4.6000e+02,
          5.6000e+02],
         [7.0000e+01, 1.7000e+02, 2.7000e+02, 3.7000e+02, 4.7000e+02,
          5.7000e+02]],

        [[1.0000e+00, 1.0100e+02, 2.0100e+02, 3.0100e+02, 4.0100e+02,
          5.0100e+02],
         [1.1000e+01, 1.1100e+02, 2.1100e+02, 3.1100e+02, 4.1100e+02,
          5.1100e+02],
         [4.1000e+01, 1.4100e+02, 2.4100e+02, 3.4100e+02, 4.4100e+02,
          5.4100e+02],
         [6.1000e+01, 1.6100e+02, 2.6100e+02, 3.6100e+02, 4.6100e+02,
          5.6100e+02],
         [7.1000e+01, 1.7100e+02, 2.7100e+02, 3.7100e+02, 4.7100e+02,
          5.7100e+02]],

        [[2.0000e+00, 1.0200e+02, 2.0200e+02, 3.0200e+02, 4.0200e+

$gather$ won! ;) Thanks for your attention. 